In [1]:
print('\n> Cloning the repo')
!git clone https://github.com/saic-mdal/lama.git


> Cloning the repo
Cloning into 'lama'...
remote: Enumerating objects: 383, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 383 (delta 66), reused 55 (delta 47), pack-reused 269
Receiving objects: 100% (383/383), 9.87 MiB | 14.35 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [2]:
print('\n> Install dependencies')
!pip install torch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0 torchtext==0.9
!pip install -r lama/requirements.txt --quiet
!pip install wget --quiet
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html --quiet


> Install dependencies
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 80.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1+cu116
    Uninstalling torchvision-0.14.1+cu116:
      Successfully uninstalled torchvision-0.14.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
  Attempting uninstall: torc

In [3]:
print('\n> Changing the dir to:')
%cd /content/lama


> Changing the dir to:
/content/lama


In [4]:
print('\n> Download the model')
!curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o big-lama.zip
!unzip big-lama.zip


> Download the model
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  363M    0  363M    0     0  8846k      0 --:--:--  0:00:42 --:--:-- 8447k
Archive:  big-lama.zip
  inflating: big-lama/config.yaml    
  inflating: big-lama/models/best.ckpt  


In [5]:
print('>fixing opencv')
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet

>fixing opencv
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 56.1 MB/s eta 0:00:00


In [6]:
print('\n> Init mask-drawing code')
import base64, os
import matplotlib.pyplot as plt
import numpy as np
from shutil import copyfile
import shutil

from IPython.display import HTML, Image
from base64 import b64decode
import wget


> Init mask-drawing code


In [7]:
import pandas as pd
from tqdm.auto import tqdm
from math import ceil, floor
from PIL import Image
import requests

In [8]:
def img_changer(url, fname):
  
  response = requests.get(url, stream=True)
  with open(f'{fname}', 'wb') as out_file:
      shutil.copyfileobj(response.raw, out_file)
  del response

  img = Image.open(f'{fname}').convert("RGBA")
  size = img.size
  img.save(f'{fname}', 'PNG')

  copyfile(fname, f'./data_for_prediction/{fname}')
  os.remove(fname)
  fname = f'./data_for_prediction/{fname}'

  with open('key2.txt', 'rb') as f:
    binary = f.read()
  filename=f"./{fname.split('.')[1]}_mask.png"
  with open(filename, 'wb') as f:
    f.write(binary)

  img = Image.open(f'{filename}').convert("RGBA")
  border = (580 - size[0]) / 2
  img = img.crop((floor(border), 0, 580 - ceil(border), 960))
  img.save(f'{filename}', 'PNG')

  with_mask = np.array(plt.imread(f"./{fname.split('.')[1]}_mask.png")[:,:,:3])
  mask = (with_mask[:,:,0]==1)*(with_mask[:,:,1]==0)*(with_mask[:,:,2]==0)

  plt.imsave(f"./{fname.split('.')[1]}_mask.png", mask, cmap='gray')
  #return size

Перед выполнением ячейки ниже загрузить нужную таблицу (название таблицы в переменной tablename) и файл key2.txt в папку lama.

In [25]:
#tablename = 'img_lt.xlsx' 
tablename = 'Lithuanian_wines_3.xlsx'

df = pd.read_excel(tablename)
df = df[['wine_id', 'image_vivino']].drop_duplicates()
df.loc[:, 'image_vivino'] = df['image_vivino'].apply(lambda s: s if 'x960' in s else np.NaN)
df = df.dropna().reset_index().copy()

In [26]:
shutil.rmtree('./data_for_prediction', ignore_errors=True)
! mkdir data_for_prediction

#sizes = {}

for i in tqdm(df.index):
#  url = "https:" + df.loc[i, 'img_perfect_url'] 
  url = "https:" + df.loc[i, 'image_vivino']
  wine_id = str(df.loc[i, 'wine_id'])
  fname = wine_id + '.png'
  #sizes[wine_id] = img_changer(url, fname)
  img_changer(url, fname)

!PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py model.path=$(pwd)/big-lama indir=$(pwd)/data_for_prediction outdir=/content/output  dataset.img_suffix=.png > /dev/null


  0%|          | 0/924 [00:00<?, ?it/s]

100% 924/924 [08:26<00:00,  1.82it/s]


Приведение фона изображений к прозрачному.

In [28]:
for k in tqdm(df.index):
  wine_id = str(df.loc[k, 'wine_id'])
  fname = '../output/' + wine_id + '_mask.png'
    
  img = Image.open(f'{fname}').convert("RGBA")
  datas = list(img.getdata())

  sz = len(datas)
  line = sz // 960

  newData = [0] * sz
  for i in range(sz):
    if (datas[i][0] == 76 and datas[i][1] == 105 and datas[i][2] == 113):
      newData[i] = (0, 0, 0, 0)
    else:
      newData[i] = datas[i]
        
      if (datas[i-line][0] == 76 and datas[i-line][1] == 105 and datas[i-line][2] == 113):
        newData[i] = (0, 0, 0, 0)
        newData[min(i+line, sz-1)] = (0, 0, 0, 0)
        newData[min(i+2*line, sz-1)] = (0, 0, 0, 0)
        
      if (datas[i+1][0] == 76 and datas[i+1][1] == 105 and datas[i+1][2] == 113):
        newData[i] = (0, 0, 0, 0)
        newData[i-1] = (0, 0, 0, 0)          
        newData[i-2] = (0, 0, 0, 0)

      if (datas[min(i+line, sz-1)][0] == 76 and datas[min(i+line, sz-1)][1] == 105 and datas[min(i+line, sz-1)][2] == 113):
        newData[i] = (0, 0, 0, 0)
        newData[i-line] = (0, 0, 0, 0)
        newData[i-2*line] = (0, 0, 0, 0)

      if (datas[i-1][0] == 76 and datas[i-1][1] == 105 and datas[i-1][2] == 113):
        newData[i] = (0, 0, 0, 0)
        newData[i+1] = (0, 0, 0, 0)
        newData[i+2] = (0, 0, 0, 0)

  img.putdata(newData)

  img.save(f'../output/{wine_id}.png', 'PNG')

  os.remove(fname)

  0%|          | 0/924 [00:00<?, ?it/s]

Все готовые изображения в папке /content/output (путь из папки lama: ../output).
Итого, все изображения в формате '.png' с прозрачным фоном, все одного размера (580*960, как размер самого большого изображения)

In [29]:
#@title save yo data to drive
filename = "output3" #@param {type:"string"}
folders_or_files_to_save = "../output" #@param {type:"string"}
from google.colab import files
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

def save_file_to_drive(name, path):
    file_metadata = {
    'name': name,
    'mimeType': 'application/octet-stream'
    }

    media = MediaFileUpload(path, 
                  mimetype='application/octet-stream',
                  resumable=True)

    created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    print('File ID: {}'.format(created.get('id')))

    return created


extension_zip = ".zip"

zip_file = filename + extension_zip

# !rm -rf $zip_file
!zip -r $zip_file {folders_or_files_to_save} # FOLDERS TO SAVE INTO ZIP FILE

auth.authenticate_user()
drive_service = build('drive', 'v3')

destination_name = zip_file
path_to_file = zip_file
save_file_to_drive(destination_name, path_to_file)


  adding: ../output/ (stored 0%)
  adding: ../output/1721320.png (deflated 0%)
  adding: ../output/2580011.png (deflated 0%)
  adding: ../output/1135810.png (deflated 0%)
  adding: ../output/1146198.png (deflated 0%)
  adding: ../output/1991374.png (deflated 1%)
  adding: ../output/1776414.png (deflated 0%)
  adding: ../output/2790.png (deflated 0%)
  adding: ../output/1163903.png (deflated 0%)
  adding: ../output/1135108.png (deflated 0%)
  adding: ../output/1194573.png (deflated 0%)
  adding: ../output/1523074.png (deflated 0%)
  adding: ../output/24016.png (deflated 1%)
  adding: ../output/2700140.png (deflated 0%)
  adding: ../output/7830104.png (deflated 0%)
  adding: ../output/15226.png (deflated 0%)
  adding: ../output/1438705.png (deflated 0%)
  adding: ../output/1193927.png (deflated 0%)
  adding: ../output/1434974.png (deflated 0%)
  adding: ../output/2795324.png (deflated 0%)
  adding: ../output/1244860.png (deflated 0%)
  adding: ../output/19548.png (deflated 0%)
  adding: 

{'id': '1SI5a5iH8l_1fVLWQeN0lbnEqsVokU6pb'}